In [13]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# import datetime as dt

## Procesamiento de datos

In [3]:
dataset_sube = pd.read_csv('data/sube-2023.csv')

### Ejercicio 1

#### a)

In [5]:
dataset_sube['DIA_TRANSPORTE'] = pd.to_datetime(dataset_sube['DIA_TRANSPORTE'], format='%Y-%m-%d')

#### b)

##### i.

In [8]:
def obtener_nombre_dia(fecha_dia):
    dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
    return dias[fecha_dia.weekday()]

In [9]:
dataset_sube['FECHA_DIA'] = dataset_sube['DIA_TRANSPORTE'].apply(obtener_nombre_dia)

##### ii.

In [16]:
dataset_sube['FECHA_ORDINAL'] = dataset_sube['DIA_TRANSPORTE'].dt.day_of_year

##### iii.

### Ejercicio 2

### Ejercicio 3

## Analisis exploratorio

### Ejercicio 4

## Modelado

### Ejercicio 5